In [1]:
# this mounts your Google Drive to the Colab VM.
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

# enter the foldername in your Drive where you have saved the unzipped
# assignment folder, e.g. 'cs231n/assignments/assignment3/'
FOLDERNAME = "mango_class/"
assert FOLDERNAME is not None, "[!] Enter the foldername."

# now that we've mounted your Drive, this ensures that
# the Python interpreter of the Colab VM can load
# python files from within it.
import sys
sys.path.append('/content/drive/My Drive/{}'.format(FOLDERNAME))

# this downloads the CIFAR-10 dataset to your Drive
# if it doesn't already exist.
%cd drive/My\ Drive/$FOLDERNAME/

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive
/content/drive/My Drive/mango_class


In [2]:
import torch
USE_GPU = True

# dtype = torch.float32 # we will be using float throughout this tutorial

if USE_GPU and torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

# Constant to control how frequently we print train loss
print_every = 100

print('using device:', device)



using device: cuda


In [3]:
import os
import cv2
from PIL import Image#https://yungyuc.github.io/oldtech/python/python_imaging.html
import torchvision
import matplotlib.pyplot as plt

from torch.utils.data import DataLoader
from torch.utils.data import sampler
import numpy as np
from PIL import Image#https://yungyuc.github.io/oldtech/python/python_imaging.html




In [4]:
from py_files.dataloader2 import *
# import random as rd

In [5]:
dataloaders = {"dev":None}
dataset_sizes = {"dev":None}
devset = mango(
    pic_root='/C1-P1_Train_Dev_fixed_nobatch/C1-P1_Dev/C1-P1_Dev_all',
    label_root = '/C1-P1_Train_Dev_fixed_nobatch/dev.csv',
    preload=True,
    phase = "dev"
)#把traning的dataset載下來



In [6]:
batch_size = 128

devset_loader = DataLoader(devset, batch_size=batch_size
                            , shuffle=False, num_workers=4)

# Use the torch dataloader to iterate through the dataset

# dataloaders.update(train = trainset_loader)
dataloaders.update(dev = devset_loader)
# dataset_sizes.update(train = len(trainset_loader))
dataset_sizes.update(dev = len(devset_loader))

In [7]:
#in this cell, you can check if you had already finished preprocessing
#if ok: it will show 800
#else: it will show 0
# path = './C1-P1_Train_Dev_fixed/C1-P1_Train/batch' + str(i)
# path2 = path.replace("Train/","Train_after/")
# train_after_files = os.listdir(path2)
# print(len(train_after_files))

In [8]:
# devset = mango(
#     pic_root='/C1-P1_Train_Dev_fixed/C1-P1_Dev',
#     label_root = '/C1-P1_Train_Dev_fixed/dev.csv',
#     preload=True,
#     phase = "dev",
# )#把dev的dataset載下來

# devset_loader = DataLoader(devset, batch_size=batch_size
#                             , shuffle=True, num_workers=4)
# # Use the torch dataloader to iterate through the dataset

# dataloaders.update(train = trainset_loader)
dataloaders.update(dev = devset_loader)
# dataset_sizes.update(train = len(trainset_loader))
dataset_sizes.update(dev = len(devset_loader))


In [11]:
import torchvision.models as models
import torch.nn as nn
import torch.nn.functional as F
# from sklearn.svm import LinearSVC as svm
# from sklearn.model_selection import cross_validate
from sklearn.externals import joblib
import torch.optim as optim
from torch.optim import lr_scheduler
import time
import copy

/usr/local/lib/python3.6/dist-packages/sklearn/externals/joblib/__init__.py:15: FutureWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=FutureWarning)


In [12]:

model1 = models.resnet152(pretrained=True)
model1 = nn.Sequential(*list(model1.children())[:-1])

for param in model1.parameters():
    param.requires_grad = False

model1 = model1.to(device)
##############################################################################
model2 = models.vgg16(pretrained=True)
model2 = nn.Sequential(*list(model2.children())[:-1])
model2[1] = nn.AdaptiveAvgPool2d(output_size=(1, 1))

for param in model2.parameters():
    param.requires_grad = False

model2 = model2.to(device)
##############################################################################
model3 = models.googlenet(pretrained=True)
model3 = nn.Sequential(*list(model3.children())[:-2])

for param in model3.parameters():
    param.requires_grad = False

model3 = model3.to(device)
###############################################################################
model4 = models.densenet121(pretrained=True)
model4 = nn.Sequential(*list(model4.children())[:-1])
model4 = nn.Sequential(
    *list(model4[0].children())[:-1],
    nn.AdaptiveAvgPool2d(output_size=(1, 1)),
    )

for param in model4.parameters():
    param.requires_grad = False

model4 = model4.to(device)

Downloading: "https://download.pytorch.org/models/resnet152-b121ed2d.pth" to /root/.cache/torch/checkpoints/resnet152-b121ed2d.pth


Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/checkpoints/vgg16-397923af.pth


Downloading: "https://download.pytorch.org/models/googlenet-1378be20.pth" to /root/.cache/torch/checkpoints/googlenet-1378be20.pth


Downloading: "https://download.pytorch.org/models/densenet121-a639ec97.pth" to /root/.cache/torch/checkpoints/densenet121-a639ec97.pth


In [13]:
def test_model(loader):
  count = 0
  scores_true = torch.zeros([0, 4608]).to(device)

  labels = torch.zeros(0, dtype = torch.long).to(device)

  # model1.fc.load_state_dict(torch.load("model_params/nobatch1_resnet152_try.pth"))
  # model2.fc.load_state_dict(torch.load("model_params/nobatch2_vgg16_try.pth"))
  # model3.fc.load_state_dict(torch.load("model_params/nobatch3_googlenet_try.pth"))
  # model4.classifier.load_state_dict(torch.load("model_params/nobatch4_densenet121_try.pth"))
  # try:
  # model.fc.load_state_dict(torch.load("model_params/nobatch" + str(num_model) + "_" + which_net + ".pth"))
  # except:
  
  # num_correct = 0
  num_samples = 0
  model1.eval()  # set model to evaluation mode
  model2.eval()
  model3.eval()
  model4.eval()
  with torch.no_grad():
      for x, y in loader:
          x = x.to(device=device, dtype=dtype)  # move to device, e.g. GPU
          y = y.to(device=device, dtype=torch.long)

          scores1 = model1(x)
          scores2 = model2(x)
          scores3 = model3(x)
          scores4 = model4(x)

          for i in [-1, -1]:
            scores1 = scores1.squeeze(i)
            scores2 = scores2.squeeze(i)
            scores3 = scores3.squeeze(i)
            scores4 = scores4.squeeze(i)

          scores_all = torch.cat((scores1, scores2, scores3, scores4), 1)#(128,4608)
          scores_true = torch.cat((scores_true, scores_all), 0)#(1400, 4608)
          
          labels = torch.cat((labels, y), 0)
         
  return scores_true, labels


In [14]:
feat_extra, labels = test_model(devset_loader)

In [15]:
torch.save(feat_extra, "model_params/dev_feat_extra.pth")
torch.save(labels, "model_params/dev_label.pth")

In [16]:
loal = torch.load(open("./model_params/dev_feat_extra.pth", 'rb'))

In [18]:
loal.size()

torch.Size([800, 4608])

In [ ]:
# loal = torch.load(open("./model_params/nobatch"+str(i)+"_resnet152_try.pth", 'rb'))
# dic = loaded_model['svm_model']
# res = dic.score(inp, labels)
# loal["1.weight"]